In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.append(os.path.join(project_root, "src"))

In [ ]:
import pytorch_lightning as pl
import torch
import monai
from monai.networks.nets import SegResNet, UNETR
from monai.losses import DiceCELoss


class SegResModel(pl.LightningModule):
    def __init__(self, in_channels, out_channels, learning_rate=1e-3):
        super(SegResModel, self).__init__()
        # self.model = UNet(
        #     spatial_dims=3,
        #     in_channels=in_channels,
        #     out_channels=out_channels,
        #     channels=(16, 32, 64, 128, 256),
        #     strides=(2, 2, 2, 2),
        #     num_res_units=2,
        # )

        self.model = SegResNet(spatial_dims=3, in_channels=2, out_channels=4)
        self.loss_fn = DiceCELoss(to_onehot_y=True, softmax=True)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.loss_fn(outputs, labels)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

/home/tmb2k01/deep_learning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class EnsembleModel(pl.LightningModule):
    def __init__(self, model_list, num_classes):
        super(EnsembleModel, self).__init__()
        self.models = model_list
        self.num_classes = num_classes

    def forward(self, x):
        # Collect predictions from each model in the ensemble
        predictions = [model(x) for model in self.models]
        # Average predictions
        averaged_prediction = torch.mean(torch.stack(predictions), dim=0)
        return averaged_prediction

In [4]:
from dataloader import BrainTumourDataModule

data_module = BrainTumourDataModule("../data/BrainTumourData/imagesTr/")
data_module.prepare_data()
data_module.setup()

In [ ]:
# Instantiate each model
model1 = SegResModel(in_channels=2, out_channels=4)
model2 = SegResModel(
    in_channels=2, out_channels=4
)  # replace with other architectures, e.g., UNETR

# Train each model separately
trainer = pl.Trainer(max_epochs=50)
trainer.fit(model1, data_module)  # data_module is your existing BrainTumourDataModule
trainer.fit(model2, data_module)

# Create the ensemble model using the trained models
ensemble_model = EnsembleModel([model1, model2], num_classes=4)

# Evaluate on validation or test data
ensemble_predictions = trainer.predict(
    ensemble_model, dataloaders=data_module.test_dataloader()
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/tmb2k01/deep_learning/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/tmb2k01/deep_learning/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_

Epoch 0:   0%|          | 0/328 [00:00<?, ?it/s] 

RuntimeError: The size of tensor a (40) must match the size of tensor b (39) at non-singleton dimension 4